# 1 Introduction
## 1.1 Aim
We would like to explore an ASX-listed company and how certain external factors influence its share price. In this report, we will step through the data obtaining and cleaning process that will later enable us to perform the relevant analysis.

The company we will be looking at is **Costa Group (CGC)**, one of Australia's leading growers of fresh fruit and vegetables. We wish to analyse how **weather conditions** around their farms and **announcements** by their company influence their share price. The time period which we will obtain and prepare data for will be from CGC's ASX listing date (24$^{th}$ July 2015) to the 1$^{st}$ October 2019.

## 1.2 Data Collection
For our first data set, we need to obtain the historical share prices for CGC. These are conveniently supplied [here](https://au.finance.yahoo.com/quote/CGC.AX/history/) by Yahoo Finance. We can adjust the time period and click 'Download Data' as shown below:

<img src="https://imgur.com/qu2FNpt.png" width="700">


The downloaded file is of .csv format. It contains a header row with 7 attributes and 1062 other rows, with each row representing the key share price and volume observations for a single day.

<br>

The second type of data we require is weather data around CGC's farms. Acquiring this data is more complex than our first data set was. Firstly, we need to take a look at CGC's latest financial report to find out where their farms are located. Page 10 of their 2018 six-monthly financial period [report](http://investors.costagroup.com.au/FormBuilder/_Resource/_module/YfnrttzbYEyUJyNrb86SEg/file/report/Six-month_financial_period_2018.pdf), contains this nice visualisation of all of their farms:

<img src="https://imgur.com/YSVUaLv.png" width="700">

From all of the locations listed above, we are only interested in those that have 'farm' in their name (and also 'Tomato Glasshouse, Guyra). We also do not want to collect weather data for mushroom farms because mushrooms are grown in dark and enclosed areas. We will further simplify our data collection process by excluding the international farms in China and Morocco because the report states that only 1% of CGC's total revenue is generated from international operations (also on page 10).

We now need to obtain all of the relevant weather data. The Australian Bureau of Meteorology collects daily weather data from many different weather stations around Australia. We will only collect daily rainfall and maximum temperature data as these two measurements are the main influencers of crop growth at farms.

Once at their [website](http://www.bom.gov.au/climate/data/stations/), we need to go through the following steps with all of the farm locations:
1. Type the town of the farm into the place name (e.g. Walkamin for “Berry Farm, Walkamin”).
2. Select the correct town after clicking “Find place names”.
3. Choose “Temperature - maximum” for weather element, and “Daily” for reporting frequency.
4. Click “Request or download data”.
5. Sort the table by “%”. Try to find a station that is within 50 km with >90% filled records and coverage over the 2015-present time period.
6. Download the data and note down the name of that station.
7. Now with that same station, obtain its “Rainfall - total” data.
8. Move on to the next farm and repeat.

For each station and weather measurement type, data is stored in .csv format and contained in a zipped folder. The .csv file contains a header row with 8 attributes and anywhere between 1000 and 50,000 other rows depending on the station, with each row containing information about the station and weather measurement value for a single day. In total, there are 30 .csv files (15 for rain and 15 for temperature, some close-together farms map to the same station).

Carrying out all of the above steps should result in the Farm-Station mapping described by the table below:

<table style="font-size:100%">
    <tr>
        <th> Farm </th>
        <th> Station </th>
    </tr>
    <tr>
        <td> Berry Farm, Gingin </td><td> Gingin Aero </td>
    </tr>
    <tr>
        <td> Berry Farms, Tolga <br> Berry Farm, Atherton <br> Banana Farm, Walkamin 
        <br> Berry Farm, Atherton <br> Avocado Farm, Atherton </td><td> Walkamin Research Station </td>
    </tr>
    <tr>
        <td> Banana Farm, Tully </td><td> Cardwell Marine Pde </td>
    </tr>
    <tr>
        <td> Grape Farm, Mundubbera </td><td> Gayndah Airport </td>
    </tr>
    <tr>
        <td> Avocado Farm, Childers </td><td> Bundaberg Aero </td>
    </tr>
    <tr>
        <td> Berry Farm, Corindi </td><td> Coffs Harbour Airport </td>
    </tr>
    <tr>
        <td> Tomato Glasshouse, Guyra </td><td> Guyra Hospital </td>
    </tr>
    <tr>
        <td> Berry Farm, Tumbarumba <br> Berry Farm, Rosewood </td><td> Tumbarumba Post Office </td>
    </tr>
    <tr>
        <td> Colignan Citrus Farm </td><td> Mildura Airport </td>
    </tr>
    <tr>
        <td> Yandilla Citrus Farm and Packhouse, Renmark <br> Pike Creek Farm, Lyrup 
        <br> Amaroo Citrus Farm, Murtho <br> Kangara Citrus Farm and Packhouse, Murtho </td><td> Renmark Aero </td>
    </tr>
    <tr>
        <td> Solora Citrus Farm, Loxton </td><td> Loxton Research Centre </td>
    </tr>
    <tr>
        <td> Berry Farm, Sulphur Creek </td><td> Wynyard Airport </td>
    </tr>
    <tr>
        <td> Berry Farm, Wesley Vale <br> Berry Farm, East Devonport </td><td> Devonport Airport </td>
    </tr>
    <tr>
        <td> Berry Farm, Dunorlan </td><td> Sheffield School Farm </td>
    </tr>
    <tr>
        <td> Berry Farm, Lebrina </td><td> Launceston (Ti Tree Bend) </td>
    </tr>
</table>

<br>

The final type of data we require are CGC's official company announcements. These are available as navigable tables on many different websites, but the one on Commonwealth Securities' [website](https://www2.commsec.com.au/quotes/?stockCode=CGC#/) (n.b. you need to log in with a CommSec trading account) can be customised to display an exhaustive list of CGC's announcements on one scrollable page. This allows us to carry out a quick web-scrape of the table (in Google Chrome) like so:
1. Press F12 to bring up the HTML DevTools panel.
2. Click the element inspector tool on the top-left of the panel.
3. Capture the table as shown in the picture below.
4. Copy the `<tbody data-v-0ea6aa3f>...</tbody>` section and paste it into a .txt file.
5. Delete `<tbody data-v-0ea6aa3f="">` from the first line of the .txt file and delete `</tbody>` from the end of the last line of the .txt file.

<img src="https://imgur.com/VgdkOlw.png" width="1000">

Unlike the other data files we obtained, our CGC announcements file is a .txt file of HTML code. The file contains no header row (we will add this in during our data cleaning) but has 583 other rows. Each row consists of 5 attributes detailing the title of the announcement document along with time, date, number of pages and whether or not the announcement is market sensitive (this will be an important attribute for later analysis). The attribute values in each row are divided by `<td></td>` tags, and we will use this feature to our advantage later in our data cleaning.

## 1.3 Data Usage Rights
The share price data and announcements data are both sourced from the ASX even though they are displayed by Yahoo Finance and CommSec on their websites. The ASX [Terms of Use](https://www.asx.com.au/about/terms-use.htm) states that 
> *You must not use the Content for commercial purposes without first obtaining the express written authority of ASX. Use of the Content for a commercial purpose is any use other than accessing and using the content for your own personal and private decision making.*

... where "Content" is defined as
> *all information, text, materials, graphics, software, tools, results derived from the use of software and tools, advertisements, names, logos and trade marks on the Site*

Since our data is ultimately sourced with on-site tools and information tables, and we are strictly only using the data for personal reasons, we are allowed to use the data we have sourced.

As for the weather data, the Bureau of Meteorology's data is protected by the Copyright Act 1968. We are allowed to use the data non-commercially as long as we have referenced where we sourced the data from (see Section 4 of this report).

# 2 Data Cleaning and Transformation
## 2.1 Preliminary Code
Before we begin our file cleaning and transformation process, we need to import any modules and define any functions that we will be using during the process.

In [99]:
import csv # read and write to .csv files
import glob # get names of files in a directory
import datetime as dt # work with date attributes

The `find_bad_values` function below will be used on each attribute of each resultant file after the cleaning process to check if there remain any missing or out-of-range values. Given a file name, column number and an expected range (only for numeric attributes), the function will print out any rows containing bad values under the specified column.

In [100]:
def find_bad_values(file_name, column_no, minimum='no minimum', maximum='no maximum'):
    data = list(csv.reader(open(file_name)))
    for row in data[1:]: # assume there's a header row
        if row[column_no] != '':
            if minimum != 'no minimum' and maximum != 'no maximum':
                try:
                    if float(row[column_no]) < minimum or float(row[column_no]) > maximum:
                        print(row) # out-of-range value found
                except:
                    print(row) # float conversion failure
        else:
            print(row) # missing value found

The `view_data` function below allows data that is in list-form to be displayed concisely. It takes the name of the list as input and prints out the first four and last three elements of the list. Usually, the first element of the list will be its header row.

In [101]:
def view_data(list_name):
    for row in list_name[:4]: print(row)
    print('  ⋮')
    for row in list_name[-3:]: print(row)

## 2.2 Share Price Data 
With the help of our imported `csv` module, we will read in our .csv file of share prices and store it as a list in `share_price_data_raw`.

In [102]:
share_price_data_raw = list(csv.reader(open('CGC.AX.csv')))
view_data(share_price_data_raw)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2015-07-24', '2.240000', '2.270000', '2.160000', '2.160000', '1.972063', '23179807']
['2015-07-27', '2.190000', '2.190000', '2.160000', '2.160000', '1.972063', '2417254']
['2015-07-28', '2.150000', '2.160000', '2.010000', '2.040000', '1.862504', '2379886']
  ⋮
['2019-09-27', '3.600000', '3.645000', '3.530000', '3.530000', '3.530000', '2260278']
['2019-09-30', '3.530000', '3.795000', '3.510000', '3.750000', '3.750000', '2826881']
['2019-10-01', '3.750000', '3.770000', '3.650000', '3.730000', '3.730000', '1910395']


The data is fairly clean as it is. Notice that the rows are listed chronologically, starting from CGC's ASX listing date of 2015-07-24 (already in the ISO 8601 standards format). Dates such as 2015-07-25 are missing because they are either weekends or national holidays; the ASX share market does not operate during these days. We do not need to do anything about that.

However, we would like to reformat the price-associated attributes so that they better represent dollar amounts, i.e. to two decimal places and right-padded with zeroes where required. We will define a function that carries out this conversion below.

In [103]:
def monetary_display(value):
    price = str(round(float(value), 2))
    if len(price.split('.')[1]) == 1:
        price += '0'
    return str(price)

We would also like to remove the `Adj Close` attribute from the data because it is just a more complex version of the already-existing `Close` attribute and we would like to keep things simple for future analysis. Hence, we will develop a cleaned data set and store it in the list `share_price_data_final`.

In [104]:
share_price_data_final = [['Date', 'Open', 'High', 'Low', 'Close', 'Volume']] # header row

for row in share_price_data_raw[1:]:
    date = row[0]
    open_price = monetary_display(row[1])
    high_price = monetary_display(row[2])
    low_price = monetary_display(row[3])
    close_price = monetary_display(row[4])
    volume = row[6]
    share_price_data_final.append([date, open_price, high_price, low_price, close_price, volume])
    
view_data(share_price_data_final)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
['2015-07-24', '2.24', '2.27', '2.16', '2.16', '23179807']
['2015-07-27', '2.19', '2.19', '2.16', '2.16', '2417254']
['2015-07-28', '2.15', '2.16', '2.01', '2.04', '2379886']
  ⋮
['2019-09-27', '3.60', '3.65', '3.53', '3.53', '2260278']
['2019-09-30', '3.53', '3.79', '3.51', '3.75', '2826881']
['2019-10-01', '3.75', '3.77', '3.65', '3.73', '1910395']


The `share_price_data_final` list can now be written to a .csv file. Let's also view the data:

In [105]:
with open('Share_Price_Data.csv', 'w', newline='\n') as myfile:
    for entry in share_price_data_final:
        csv.writer(myfile).writerow(entry)
        
view_data(share_price_data_final)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
['2015-07-24', '2.24', '2.27', '2.16', '2.16', '23179807']
['2015-07-27', '2.19', '2.19', '2.16', '2.16', '2417254']
['2015-07-28', '2.15', '2.16', '2.01', '2.04', '2379886']
  ⋮
['2019-09-27', '3.60', '3.65', '3.53', '3.53', '2260278']
['2019-09-30', '3.53', '3.79', '3.51', '3.75', '2826881']
['2019-10-01', '3.75', '3.77', '3.65', '3.73', '1910395']


Finally, we will perform a scan of all of the attributes of the new file to ensure that there are no bad values using our `find_bad_values` function. If there is nothing printed out after the code, then we are finished.

In [106]:
find_bad_values('Share_Price_Data.csv', 0) # Date
for column in range(1, 5): find_bad_values('Share_Price_Data.csv', column, 0, 50) # Open, High, Low, Close
find_bad_values('Share_Price_Data.csv', 5, 0, 10e7) # Volume

## 2.3 Weather Data
The cleaning and transformation process for our weather data will be a lot more involved because we have to combine 30 different .csv files into one.

We will begin by extracting all of the dates in our `share_price_data_final` list from before into another list called `asx_dates`. When we go to process our weather data files, we can check against the `asx_dates` list to make sure we only keep weather observations between the 24$^{th}$ July 2015 to the 1$^{st}$ October 2019 on days when the ASX share market operates.

In [107]:
asx_dates=[]
for row in share_price_data_final[1:]:
    asx_dates.append(row[0])

view_data(asx_dates)

2015-07-24
2015-07-27
2015-07-28
2015-07-29
  ⋮
2019-09-27
2019-09-30
2019-10-01


Before going further, let's look at one of the weather data files to get an idea of the structure of the data:

In [108]:
loxton_rainfall = list(csv.reader(open('Weather Files/Loxton_rain.csv')))
view_data(loxton_rainfall)

['Product code', 'Bureau of Meteorology station number', 'Year', 'Month', 'Day', 'Rainfall amount (millimetres)', 'Period over which rainfall was measured (days)', 'Quality']
['IDCJAC0009', '024024', '1984', '01', '01', '', '', '']
['IDCJAC0009', '024024', '1984', '01', '02', '', '', '']
['IDCJAC0009', '024024', '1984', '01', '03', '', '', '']
  ⋮
['IDCJAC0009', '024024', '2019', '09', '30', '0.0', '1', 'N']
['IDCJAC0009', '024024', '2019', '10', '01', '0.0', '1', 'N']
['IDCJAC0009', '024024', '2019', '10', '02', '0.0', '1', 'N']


There are a few things we would like to do regarding the attributes of the data set. We want to replace the `Bureau of Meteorology station number` attribute with the name of the station it represents and the state that the station is located in. We can create a dictionary relating each station number (for all the stations that we have .csv files) to its name and state like so:

In [109]:
#dictionary with key=stationnumber, value=['stationname','state']
number_name = {
    9178 : ["Gingin Aero","WA"],
    31108 : ["Walkamin Research Station","QLD"],
    32004 : ["Cardwell Marine Pde","QLD"],
    39066 : ["Gayndah Airport","QLD"],
    39128 : ["Bundaberg Aero","QLD"],
    59151 : ["Coffs Harbour Airport","NSW"],
    56229 : ["Guyra Hospital","NSW"],
    72043 : ["Tumbarumba Post Office","NSW"],
    91107 : ["Wynyard Airport","TAS"],
    91126 : ["Devonport Airport","TAS"],
    91291 : ["Sheffield School Farm","TAS"],
    76031 : ["Mildura Airport","VIC"],
    24048 : ["Renmark Aero","SA"],
    24024 : ["Loxton Research Centre","SA"],
    91237 : ["Launceston (Ti Tree Bend)", "TAS"]
}

We would also like to merge and delete some attributes. The `Product code`, `Period over which rainfall was measured (days)` and `Quality` are not very useful to us, so we will remove them. The `Year`, `Month` and `Day` should be merged into one attribute that showcases the date in ISO 8601 standard format.

We will do a similar thing with the temperature files, and also remove the `Days of accumulation of maximum temperature` attribute in those files.

Upon observing the rows in the rainfall and temperature files, we notice that some of them are missing values for the `Total Rainfall (mm)` and `Max Temperature (degrees Celsius)` attributes. We would like to fill in these missing values with the average of the most recent day that had a measurement and the next closest day in the future that has a measurement. For example:

<img src="https://imgur.com/EpbYlqj.png" width="800">

Finally, we would like to join the `Max Temperature (degrees Celsius)` attribute of each station's temperature file onto its corresponding station's rainfall, and then combine all of the 15 resultant files into one big final .csv file.

All of the procedures we wish to do mentioned above are carried out in the following chunk of code:

In [110]:
weather_data_final = [["Station Name", "State", "Date", "Total Rainfall (mm)", "Max Temperature (degrees Celsius)"]] # header
directory = list(glob.iglob('Weather Files/*.csv')) # get all .csv files in the Weather Files directory
directory.sort()

for file in directory[::2]:
    
    # at each go, input the rainfall csv file max_temp csv file for the same station
    # e.g. Walkamin_rain.csv Walkamin_temp.csv
    rainfile=file
    tempfile=directory[directory.index(file) + 1]

    #two for-loops for rain csv file. two for-loops for temp file
    no_rain_value={}
    no_temp_value={}
    rainfall_lines=[]

    f=list(csv.reader(open(rainfile)))
    g=list(csv.reader(open(tempfile)))

    #first for-loop finds missing entries in rainfall file and alocates them values using the mean of non-empty adjacent entries
    i=1
    while i<len(f):
        line_list=f[i]
        rainfall=line_list[5]
        if rainfall=="":# locate an empty value
            #locate the value before which is not empty
            # since integers are immutable, if we assign i to something, then change that someting, i is not changed
            year=line_list[2]
            month=line_list[3]
            day=line_list[4]
            ISOdate=year+"-"+month.zfill(2)+"-"+day.zfill(2)

            left=None
            right=None
            j=i-1
            #loop backwards to get a value for the top side to average
            while j>=1:
                if f[j][5]!='':
                    left=float(f[j][5])
                    break
                j-=1
            #this method only works if the very first data value is filled.

            #loop forward to get a value for the bottom value to average
            k=i+1
            while k<len(f):# only works if the last data value is filled
                if f[k][5]!='':
                    right=float(f[k][5])
                    break
                k+=1
            if left!=None and right!=None:
                # if either the left value or the right value cannot be initialised, we dont add the date to the dictionary.
                #the result is, that the missing day will have an empty string in the final dataset.
                average=round((left+right)/2, 1)

                # each date is unique so there will be no such thing as repeats in the dictionary
                no_rain_value[ISOdate]=average
        i+=1

    # second for-loop. this appends strings to rainfall_lines. each string is a line to be appended to the final file.
    # each string is in the form:'station_name,state,ISOdate,daily rainfall (mm)'
    is_first_line=True
    station_name=None
    for line in f:
        if is_first_line:
            is_first_line=False
        else:
            line_list = line
            for number in number_name:
                if number==int(line_list[1]):
                    station_name=number_name[number][0]
                    state=number_name[number][1]
            assert station_name!=None

            rainfall=line_list[5]
            ISOdate=line_list[2]+"-"+line_list[3].zfill(2)+"-"+line_list[4].zfill(2)
            ISOdate=ISOdate.strip()

            if ISOdate in asx_dates:
                if ISOdate in no_rain_value:
                    line_to_append=station_name+','+state+','+ISOdate+','+str(no_rain_value[ISOdate])
                    rainfall_lines.append(line_to_append)
                else:
                    line_to_append=station_name+','+state+','+ISOdate+','+str(rainfall)
                    rainfall_lines.append(line_to_append)

    # third for-loop. loop through the max_temp file for missing entries. allocates them values using the mean of non-empty
    # adjacent entries
    i=1
    while i<len(g):
        line_list=g[i]
        temp=line_list[5]
        if temp=='': # locate a missing value

            year=line_list[2]
            month=line_list[3]
            day=line_list[4]
            ISOdate=year+"-"+month.zfill(2)+"-"+day.zfill(2)

            left=None
            right=None
            j=i-1
            #loop backwards to get a value for the top side to average
            while j>=1:
                if g[j][5]!='':
                    left=float(g[j][5])
                    break
                j-=1
            #this method only works if the very first data value is filled.

            #loop forward to get a value for the bottom value to average
            k=i+1
            while k<len(g):# only works if the last data value is filled
                if g[k][5]!='':
                    right=float(g[k][5])
                    break
                k+=1

            if left!=None and right!=None:
                # if either the left value or the right value cannot be initialised, we dont add the date to the dictionary.
                #the result is, that the missing day will have an empty string in the final dataset.
                average=round((left+right)/2, 1)
                no_temp_value[ISOdate]=average
        i+=1

    # fourth for-loop. this one appends to weather_data_final
    is_first_line=True
    for line in g:
        if is_first_line:
            is_first_line=False
        else:
            line_list=line
            ISOdate=line_list[2]+"-"+line_list[3].zfill(2)+"-"+line_list[4].zfill(2)
            ISOdate=ISOdate.strip()

            if ISOdate in asx_dates:
                if ISOdate in no_temp_value:
                    #search for the corresponding rainfall line
                    temp=no_temp_value[ISOdate]
                    for string in rainfall_lines:
                        if ISOdate in string:
                            line=string+","+str(temp)
                            weather_data_final.append(line.split(","))
                else:
                    temp=float(line_list[5])
                    for string in rainfall_lines:
                        if ISOdate in string:
                            line=string+","+str(temp)
                            weather_data_final.append(line.split(","))

The `weather_data_final` list holds the final data that we can now write to a .csv file. Let's also take a glimpse at the `weather_data_final` list:

In [111]:
with open('Weather_Data.csv', 'w', newline='\n') as myfile:
    for entry in weather_data_final:
        csv.writer(myfile).writerow(entry)
                   
view_data(weather_data_final)

['Station Name', 'State', 'Date', 'Total Rainfall (mm)', 'Max Temperature (degrees Celsius)']
['Bundaberg Aero', 'QLD', '2015-07-24', '0.2', '26.5']
['Bundaberg Aero', 'QLD', '2015-07-27', '0.0', '26.4']
['Bundaberg Aero', 'QLD', '2015-07-28', '0.0', '23.2']
  ⋮
['Wynyard Airport', 'TAS', '2019-09-27', '0.0', '14.4']
['Wynyard Airport', 'TAS', '2019-09-30', '0.0', '17.1']
['Wynyard Airport', 'TAS', '2019-10-01', '0.0', '19.1']


Finally, we will perform a scan of all of the attributes of our new Weather_Data.csv file to ensure that there are no missing or out-of-range values using our `find_bad_values` function. If there is nothing printed out after the code, then we are finished.

In [112]:
for column in range(0,3): find_bad_values('Weather_Data.csv', column) # Station Name, State, Date
find_bad_values('Weather_Data.csv', 3, 0, 1000) # Rainfall
find_bad_values('Weather_Data.csv', 4, -20, 60) # Max Temperature

## 2.4 Announcements Data
The way we will go about cleaning our announcements data will be different yet again. Firstly, we will see how the data looks in the .txt file we obtained earlier:

<img src="https://imgur.com/rU8PIjv.png" width="800">

The rows are not actually broken up properly and there are many tab characters amongst the data that we do not want. As a result, we will instead remove all tab `\t` and newline `\n` characters from the data (essentially making it one big block of text) and split it back into rows on the `</tr>` tags (this is how table rows are naturally defined in HTML). We will carry out this procedure and read the data into a list simultaneously:

In [113]:
announcements_data_raw = open('Announcements_CommSec.txt').read().replace('\t', '').replace('\n', '').split('</tr>')[:-1]
print(announcements_data_raw[0]) # we are not using our view_data() function because otherwise the output becomes very messy

<tr data-v-f1feab4e=""><td data-v-f1feab4e="">17/09/2019</td> <td data-v-f1feab4e="">4:43 pm AEST</td> <td data-v-f1feab4e=""><a data-v-f1feab4e="" href="https://commsec.api.markitondemand.com/commsec-node-api/1.0/event/document/1410-02147946-56P2ILQ5KQ6N7DVPVGH8U1U219/pdf?access_token=1e1dH4OusefhjY5YejYoO4C5EoDT" target="_blank">Change in substantial holding</a></td> <td data-v-f1feab4e="" class="text-center">7</td> <td data-v-f1feab4e="" class="text-center"><!----></td>


If we look closely, we can see that the bits of data we are interested in are surrounded by `<td...></td>` tags (these define cells in a row in HTML). We will exploit this characteristic to extract the data we require. However, even once we have done that, there is still some cleaning to do. We will need to convert the date (17/09/2019) into the ISO 8601 standard format (2019-09-17), and we will need to split the time attribute (4:43 pm AEST) into a zero-padded 24-hr time attribute (16:43) and a timezone attribute (AEST). We will carry this whole process out and store the result in the `announcements_data_final` list below:

In [114]:
announcements_data_final = []
for row in announcements_data_raw:
    fields = row.split('<td data-v-f1feab4e=""')   
    date = dt.datetime.strptime(fields[1], '>%d/%m/%Y</td> ').strftime('%Y-%m-%d') # pull date apart and glue back in ISO format
    
    # time processing
    time_raw = fields[2].replace('</td> ', '').replace('>', '')
    hour = time_raw[0:-11]
    minute = time_raw[-10:-8]
    meridiem = time_raw[-7:-5]    
    if meridiem == 'pm':
        hour = str(int(hour) + 12)
        if hour == '24':
            hour = '12'
    elif meridiem == 'am':
        if hour == '12':
            hour == '00'
    hour = hour.zfill(2)
    
    time_cleaned = hour + ':' + minute
    time_zone = time_raw[-4:]
    title = fields[3].split('target="_blank">')[1].replace('</a></td> ', '').replace('&amp;', '&')
    pages = fields[4].split('>')[1].replace('</td', '')
    market_sensitive = 'No' if fields[5].split('>')[1] == '<!----' else 'Yes'

    announcements_data_final.append([date, time_cleaned, time_zone, title, pages, market_sensitive])
    
announcements_data_final.append(['Date', '24-hr Time', 'Timezone', 'Title', 'Pages', 'Market Sensitive']) # header row
announcements_data_final.reverse()  # chronological ascending order

As we have done before, we will write the `announcements_data_final` list to a .csv file and view the data:

In [115]:
with open('Announcements_Data.csv', 'w', newline='\n') as myfile:
    for entry in announcements_data_final:
        csv.writer(myfile).writerow(entry)
                   
view_data(announcements_data_final)

['Date', '24-hr Time', 'Timezone', 'Title', 'Pages', 'Market Sensitive']
['2015-07-24', '09:54', 'AEST', 'ASX Market Release - Admission and Official Quotation', '1', 'Yes']
['2015-07-24', '09:55', 'AEST', 'ASX Market Release - Pre-quotation disclosure', '1', 'No']
['2015-07-24', '09:58', 'AEST', 'Appendix 1A and Information Form and Checklist', '20', 'No']
  ⋮
['2019-09-05', '11:38', 'AEST', 'Becoming a substantial holder', '4', 'No']
['2019-09-09', '16:40', 'AEST', 'Becoming a substantial holder', '7', 'No']
['2019-09-17', '16:43', 'AEST', 'Change in substantial holding', '7', 'No']


Finally, we will perform a scan of all of the attributes of our new Announcements_Data.csv file to ensure that there are no missing or out-of-range values using our `find_bad_values` function. If there is nothing printed out after the code, then we are finished.

In [116]:
for column in range(0, 4): find_bad_values('Announcements_Data.csv', column) # Date, 24-hr Time, Timezone, Title
find_bad_values('Announcements_Data.csv', 4, 1, 500) # Pages
find_bad_values('Announcements_Data.csv', 5) # Market Sensitive

## 2.5 Merging Datasets
In this section, we will attempt to merge our three freshly-cleaned sets of data. The key attribute that is common amongst all three data sets is the `Date` attribute. The share prices and weather data both contain entries for all ASX operating dates between 2015-07-24 and 2019-10-01, so we will simply attach each share price row to wherever there is a date match.

Adding on the announcements data is a little more difficult. Announcements are not released every single day, and sometimes there are more than one released on a single day. Hence, for each date match with a weather/share price data entry, we will attach all announcements that were released on that day, divided by '|' characters within each announcement attribute. This is a bit confusing, but it will make more sense once we view the final merged data set.

In [117]:
# initialise merged_data with all 15 attributes
merged_data = [[]]
for attribute in weather_data_final[0]: merged_data[0].append(attribute)
for attribute in share_price_data_final[0][1:]: merged_data[0].append(attribute)
for attribute in announcements_data_final[0][1:]: merged_data[0].append(attribute)
    
for row in weather_data_final[1:]:
    row = list(row)
    merged_data.append(row)
    for row_sp in share_price_data_final[1:]:
        if row_sp[0] == row[2]: # date match
            merged_data[-1].extend(row_sp[1:]) # attach share price entry when a match occurs with current weather entry
    
    merged_data[-1].extend(['','','','','']) # announcement fields
    
    for row_an in announcements_data_final[1:]:
        if row_an[0] == row[2]: # date match
            row_an = list(row_an[1:])
            for i in range(len(row_an)):
                merged_data[-1][10 + i] += str(row_an[i]) + '|' # separate announcements by '|' for matching date

view_data(merged_data)

['Station Name', 'State', 'Date', 'Total Rainfall (mm)', 'Max Temperature (degrees Celsius)', 'Open', 'High', 'Low', 'Close', 'Volume', '24-hr Time', 'Timezone', 'Title', 'Pages', 'Market Sensitive']
['Bundaberg Aero', 'QLD', '2015-07-24', '0.2', '26.5', '2.24', '2.27', '2.16', '2.16', '23179807', '09:54|09:55|09:58|09:58|09:59|10:02|10:04|10:04|10:06|10:07|10:07|10:09|10:09|', 'AEST|AEST|AEST|AEST|AEST|AEST|AEST|AEST|AEST|AEST|AEST|AEST|AEST|', 'ASX Market Release - Admission and Official Quotation|ASX Market Release - Pre-quotation disclosure|Appendix 1A and Information Form and Checklist|Constitution|Pre-quotation disclosure, Top 20, Distribution Schedule|Audited accounts full year ended 30 June 2012|Audited accounts full year ended 30 June 2013|Audited accounts full year ended 29 June 2014|Reviewed accounts half year ended 31 December 2014|Updated pro-forma statement of financial position|Employee share option plan rules|New CEO option terms and conditions|Security Trading Policy|'

The output is still a little difficult to make sense of due to the amount of attributes (15) in the merged data set. We will write the `merged_data` list to a .csv file and then it will be easier to see the final product using a spreadsheet.

In [118]:
with open('Merged_Data.csv', 'w', newline='\n') as myfile:
    for entry in merged_data:
        csv.writer(myfile).writerow(entry)

# 3 Summary Statistics
## 3.1 Preliminary Code
As we did before, we need to import any new modules and define any new functions that we will be using for processing our summary statistics.

In [119]:
from IPython.display import HTML, display # display statistics in a HTML-style table

The `calculate_aggregates` function below takes in a list of lists (the data set) and a column number. It calculates the minimum, average and maximum for the attribute specified by the column number and returns these three statistics as a list.

In [120]:
def calculate_aggregates(data, column_no):
    values = [] # define an empty list to store the column we're working with
    for row in data:
        values.append(float(row[column_no])) # add the element to our values list
      
    return [min(values), round(sum(values) / len(values), 2), max(values)] # numeric aggregates

## 3.2 Share Price Data
Recall the attributes in our `share_price_data_final` list:

In [121]:
print(share_price_data_final[0])

['Date', 'Open', 'High', 'Low', 'Close', 'Volume']


Since the list is ordered by `Date`, let's provide a simple aggregate for the `Date` attribute by printing out the most recent date.

In [122]:
print('Latest Date\n' + share_price_data_final[-1][0])

Latest Date
2019-10-01


The other attributes in the our share price data are numerical. We will use our `calculate_aggregates` function along with the imported HTML module to produce an organised table of numerical statistics for these attributes.

In [123]:
aggregates = []
for column in range(1, 6):
    aggregates.append([share_price_data_final[0][column]])
    aggregates[-1].extend(calculate_aggregates(share_price_data_final[1:], column))

header = '<tr> <th>Attribute</th> <th>Minimum</th> <th>Average</th> <th>Maximum</th> </tr>'

display(HTML(
   '<table style="font-size:100%">' + header + '<tr>{}</tr>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(value) for value in row)) for row in aggregates)
       )
    + '</table>'
))

Attribute,Minimum,Average,Maximum
Open,1.86,4.64,9.04
High,1.86,4.7,9.04
Low,1.77,4.57,8.82
Close,1.83,4.63,8.98
Volume,0.0,1493877.29,29177016.0


The table aggregates daily share price values over the 2015-07-24 to 2019-10-01 time period. The share price for CGC has ranged between \\$1.77 and $9.04, averaging around the \\$4.60 mark. The volume maxed out at almost 30 million trades in a single day, but averages at around 1.5 million per day.

## 3.3 Weather Data
Recall the attributes in our `weather_data_final` list:

In [124]:
print(weather_data_final[0])

['Station Name', 'State', 'Date', 'Total Rainfall (mm)', 'Max Temperature (degrees Celsius)']


As we did previously, we will print out the most recent date from the `Date` attribute. For the `State` attribute, we will also calculate the number of data entries for each Australian state and territory.

In [125]:
print('Latest Date\n' + weather_data_final[-1][2])

states = {'ACT':0, 'NSW':0, 'NT':0, 'SA':0, 'QLD':0, 'TAS':0, 'VIC':0, 'WA':0}
for row in weather_data_final[1:]:
    states[row[1]] += 1

print('\nEntries per State')
for key in states: print(key, states[key])

Latest Date
2019-10-01

Entries per State
ACT 0
NSW 3186
NT 0
SA 2124
QLD 4248
TAS 4248
VIC 1062
WA 1062


To make use of our last three attributes, we will produce a table of statistics for `Total Rainfall (mm)` and `Max Temperature (degrees Celsius)` grouped by `Station Name`.

In [126]:
# create dictionary where keys are stations
groupby_stations = {}
for row in weather_data_final[1:]:
    if row[0] in groupby_stations:
        groupby_stations[row[0]].append(row[1:])
    else:
        groupby_stations[row[0]] = [row[1:]]

# for both rain and temp, calculate aggregates for each station
rain_aggregates = []
temp_aggregates = []
for key in groupby_stations:
    rain_aggregates.append([key])
    rain_aggregates[-1].extend(calculate_aggregates(groupby_stations[key], 2))
    temp_aggregates.append([key])
    temp_aggregates[-1].extend(calculate_aggregates(groupby_stations[key], 3))

# calculate overall aggregates for rain and temp attributes
total_rain_aggregates = calculate_aggregates(weather_data_final[1:], 3)
total_temp_aggregates = calculate_aggregates(weather_data_final[1:], 4)

# construct a table featuring all of the aggregates
header_rain = '<tr> <th>Station</th> <th>Minimum</th> <th>Average</th> <th>Maximum</th> </tr>'
trailer_rain = '<tr> <td><b>Overall</b></td> <td><b>' 
trailer_rain += str(total_rain_aggregates[0]) + '</b></td> <td><b>'
trailer_rain += str(total_rain_aggregates[1]) + '</b></td> <td><b>'
trailer_rain += str(total_rain_aggregates[2]) + '</b></td> </tr>'

header_temp = '<tr> <th>Station</th> <th>Minimum</th> <th>Average</th> <th>Maximum</th> </tr>'
trailer_temp = '<tr> <td><b>Overall</b></td> <td><b>' 
trailer_temp += str(total_temp_aggregates[0]) + '</b></td> <td><b>'
trailer_temp += str(total_temp_aggregates[1]) + '</b></td> <td><b>'
trailer_temp += str(total_temp_aggregates[2]) + '</b></td> </tr>'

display(HTML(
   '<table style="font-size:100%">'
    + '<tr><td>Daily Rainfall (mm)</td><td>Daily Max Temperature (degrees C)</td></tr><tr><td><table>'
    + header_rain + '<tr>{}</tr>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(value) for value in row)) for row in rain_aggregates)
       )
     + trailer_rain + '</table></td>'
    
    '<td><table>' + header_temp + '<tr>{}</tr>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(value) for value in row)) for row in temp_aggregates)
       )
     + trailer_temp + '</table></td></tr></table>'
))

The table contains summary statistics for daily rainfall and temperature across our 15 selected weather stations, over the 2015-07-24 to 2019-10-01 time period. Of all stations,  Cardwell Marine Pde recorded the highest maximum rainfall in a single day, 263.0mm. As for lowest maximum daily temperature in a single day, Guyra Hospital recorded the lowest out of all stations, at 2.2 degrees C.


## 3.4 Announcements Data
Recall the attributes in our `announcements_data_final` list:

In [127]:
print(announcements_data_final[0])

['Date', '24-hr Time', 'Timezone', 'Title', 'Pages', 'Market Sensitive']


The announcements list is ordered by date and time in reverse. Let's take a look at the time and date of the first announcement and also the most recent announcement.

In [128]:
print('First Announcement:', announcements_data_final[1][0], announcements_data_final[1][1])
print('Latest Announcement:', announcements_data_final[-1][0], announcements_data_final[-1][1])

First Announcement: 2015-07-24 09:54
Latest Announcement: 2019-09-17 16:43


Next, we will find out the percentages of *AEST* and *AEDT* values under the `Timezone` attribute and also the percentages of *Yes* and *No* values under the `Market Sensitive` attribute.

In [129]:
AEST_total = 0; Yes_total = 0
for row in announcements_data_final[1:]:
    if row[2] == 'AEST': AEST_total += 1
    if row[5] == 'Yes': Yes_total += 1
        
AEST_percent = round(AEST_total / len(announcements_data_final[1:]) * 100, 2)
Yes_percent = round(Yes_total / len(announcements_data_final[1:]) * 100, 2)

print('Timezone', '\tMarket Sensitive')
print('AEST:', str(AEST_percent) + '%', '\tYes:', str(Yes_percent) + '%')
print('AEDT:', str(100 - AEST_percent) + '%', '\tNo:', str(100 - Yes_percent) + '%')

Timezone 	Market Sensitive
AEST: 52.58% 	Yes: 9.97%
AEDT: 47.42% 	No: 90.03%


There is a roughly even split between AEST and AEDT timezones. This is expected since Sydney (where the ASX operates) uses AEST for half of the year and AEDT for the other half. Furthermore, only around 10% of announcements are classified as 'Market Sensitive'.

For the `Title` attribute, let's find out what the 10 most common terms are amongst announcement titles. We will define *term* as any sequence of case-insensitive characters with no spaces in-between (these will mostly be words).

In [130]:
term_distribution = {}
for row in announcements_data_final[1:]:
    terms = row[3].split()
    for term in terms:
        if term.lower() in term_distribution:
            term_distribution[term.lower()] += 1
        else:
            term_distribution[term.lower()] = 0

print('Top 10 Terms by Frequency')
for key in sorted(term_distribution, key=term_distribution.get, reverse=True)[:10]: print(key, term_distribution[key])

Top 10 Terms by Frequency
substantial 118
a 89
holder 89
- 79
appendix 62
from 51
of 48
to 47
becoming 46
change 45


Interestingly, 'substantial' is most frequently occuring term in announcement titles, despite it being a fairly long and complicated word.

Finally, we will produce a brief statistical summary for the numeric `Pages` attribute:

In [131]:
stats = calculate_aggregates(announcements_data_final[1:], 4)
print('Number of Pages in Announcement Documents')
#print('Minimum:', stats[0], '\nAverage:', stats[1], '\nMaximum:', stats[2])
print('Minimum: \tAverage: \tMaximum:\n' + str(stats[0]) + '\t\t' + str(stats[1]) + '\t\t' + str(stats[2]))

Number of Pages in Announcement Documents
Minimum: 	Average: 	Maximum:
1.0		12.65		122.0


Announcement documents seem to average at around 12 pages. Usually, it's the annual reports that creep near the maximum of 122 pages. 

# 4 References
### Data Sources
- *Costa Group Holdings Limited (CGC.AX)*, Yahoo! Finance, https://au.finance.yahoo.com/quote/CGC.AX/history/
- *Weather Station Directory*, Australian Government: Bureau of Meteorology, http://www.bom.gov.au/climate/data/stations/
- *Login to CommSec, Commsec*, https://www2.commsec.com.au/Public/HomePage/Login.aspx?LoginResult=LoginRequired&r=https:%2f%2fwww2.commsec.com.au%2fquotes%2f%3fstockCode%3dCGC#/

### Research
- *Where We Grow*, Costa Group 2019, https://costagroup.com.au/Where-We-Grow
- *Six-month financial period 2018*, Costa Group Holdings Limited, http://investors.costagroup.com.au/FormBuilder/_Resource/_module/YfnrttzbYEyUJyNrb86SEg/file/report/Six-month_financial_period_2018.pdf
- *Growing mushrooms*, Costa Group 2019, https://costagroup.com.au/growing-mushroomsCosta 
- *Investor Centre*, Costa Group 2019, http://investors.costagroup.com.au/investor-centre/?page=asx-announcements
- *Terms of use*, ASX, https://www.asx.com.au/about/terms-use.htm

### Coding Help
- *Grok Learning DATA1002 2019 S2*, Grok Learning, https://groklearning.com/learn/usyd-data1002-2019-s2/week1/0/
- *How do I list all files of a directory?*, stackoverflow, https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
- *How do I output lists as a table in Jupyter Notebook?*, stackoverflow, https://stackoverflow.com/questions/35160256/how-do-i-output-lists-as-a-table-in-jupyter-notebook